# Accuracy Improvement of Segmentation

<br />

I am going to fit and predict a dataset of "TGS Salt Identification Challenge" on Kaggle.


https://www.kaggle.com/c/tgs-salt-identification-challenge

## [Task 1] Code Review

<br />

I will review codes of improving accuracy of segmentation by using transfer learning.

    -> clean-workflow-in-keras.ipynb
    
    -> 03-models_pretrained_and_more.ipynb

## [Task 2] Rewrite Codes

<br />

I am going to rewrite codes to change ResNet as encoder to VGG.

In [2]:
# Model is parametrized in a way to enable easy change of decoder_block type,
# as this is an argument that can be given a function, like decoder_block_simple.
def unet_resnet(input_size, decoder_block,
                # Set pretrained weights trained by Imagenet
                weights='imagenet',
                loss_func='binary_crossentropy',
                metrics_list=[my_iou_metric],
                use_lovash=False):

    # Base model - encoder
    base_model = VGG19(
        input_shape=input_size, 
        include_top=False,
        weights=weights)
    
    # Frozen layers
    for layer in base_model.layers:
        layer.trainable = True
    
    # Layers for feature extraction in the encoder part
    encoder1 = base_model.get_layer('block1_pool').output
    encoder2 = base_model.get_layer('block3_conv4').output
    encoder3 = base_model.get_layer('block4_conv4').output
    encoder4 = base_model.get_layer('block5_conv4').output
    encoder5 = base_model.get_layer('block5_pool').output

    # Center block
    center = decoder_block(
        encoder5, 'center', num_filters=512)
    concat5 = concatenate([center, encoder5], axis=-1)

    # Decoder part.
    # Every decoder block processed concatenated output from encoder and decoder part.
    # This creates skip connections.
    # Afterwards, decoder output is upsampled to dimensions equal to encoder output part.
    decoder4 = decoder_block(
        concat5, 'decoder4', num_filters=256)
    concat4 = concatenate([UpSampling2D()(decoder4), encoder4], axis=-1)

    decoder3 = decoder_block(
        concat4, 'decoder3', num_filters=128)
    concat3 = concatenate([UpSampling2D()(decoder3), encoder3], axis=-1)

    decoder2 = decoder_block(
        concat3, 'decoder2', num_filters=64)
    concat2 = concatenate([UpSampling2D()(decoder2), encoder2], axis=-1)

    decoder1 = decoder_block(
        concat2, 'decoder1', num_filters=64)
    concat1 = concatenate([UpSampling2D()(decoder1), encoder1], axis=-1)

    # Final upsampling and decoder block for segmentation.
    output = UpSampling2D()(concat1)
    output = decoder_block(
        output, 'decoder_output', num_filters=32)
    output = Conv2D(
        1, (1, 1), activation=None, name='prediction')(output)
    if not use_lovash:
        output = Activation('sigmoid')(output)
        
    model = Model(base_model.input, output)
    model.compile(loss=loss_func, optimizer='adam', metrics=metrics_list)

    return model

NameError: name 'my_iou_metric' is not defined

## [Task 3] Fit and Predict

<br />

I am going to fit and predict by using codes of ResNet and VGG, and compare the results.

    -> sprint21-google-colab.ipynb

    - The model using ResNet is the best.